In [1]:
from nltk.sentiment import SentimentIntensityAnalyzer
from googletrans import Translator

import nltk

try:
    nltk.data.find('sentiment/vader_lexicon')
except LookupError:
    nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Personal\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
from flask import Flask, request, jsonify, render_template, send_file
import pandas as pd 
import os
from io import BytesIO
import tempfile

In [3]:
app = Flask(__name__)
sia = SentimentIntensityAnalyzer()
translator = Translator()

In [4]:
def analyze_sentiment_spanish(text):
    try: 
        translation = translator.translate(text, src='es', dest= 'en')
        sentiment = sia.polarity_scores(translation.text)
        return sentiment, translation.text, None
    except Exception as e:
        print(f"Error analyzing sentiment: {e}")
        return None, None, str(e)

In [5]:
def interpret_feeling(sentiment):
    compound = sentiment['compound']

    emotion = "😐 Neutral"
    
    if compound >= 0.05:
        emotion = "😃 Positive"
    elif compound <= -0.05:
        emotion = "😒 Negative"

    category = emotion.split(" ")[1].lower()
    return emotion, category

In [6]:
def analyze_excel_file(file_path):
    """Analyze an Excel file and return the results.

    Returns a tuple `(results, error_message)` where `results` is a list of
    dictionaries with analysis for each found text cell, or `None` and an
    error string if something went wrong.
    """
    try:
        # Read the Excel file
        df = pd.read_excel(file_path)

        results = []

        # Find columns that likely contain textual content
        text_columns = []
        for column in df.columns:
            # Check if the column is object dtype and has sample text values
            if df[column].dtype == 'object' and any(isinstance(x, str) and len(x) > 10 for x in df[column].dropna()[:5]):
                text_columns.append(column)

        if not text_columns:
            return None, "No text columns found to analyze"

        # Analyze each row and text column
        for idx, row in df.iterrows():
            for column in text_columns:
                text = str(row[column]) if pd.notna(row[column]) else ""

                if len(text.strip()) > 10:  # only analyze meaningful text
                    sentiment, translation, error = analyze_sentiment_spanish(text)

                    if error is None:
                        emotion, category = interpret_feeling(sentiment)

                        results.append({
                            'row': idx + 1,
                            'column': column,
                            'original_text': text[:200] + "..." if len(text) > 200 else text,
                            'translated_text': translation,
                            'negative': f"{sentiment['neg'] * 100:.1f}%",
                            'neutral': f"{sentiment['neu'] * 100:.1f}%",
                            'positive': f"{sentiment['pos'] * 100:.1f}%",
                            'compound': f"{sentiment['compound']:.4f}",
                            'sentiment': emotion,
                            'category': category
                        })

        return results, None

    except Exception as e:
        return None, f"Error processing file: {str(e)}"

In [7]:
@app.route('/')
def index():
    """Página principal"""
    return render_template('index.html')

In [8]:
@app.route('/analyze', methods=['POST'])
def analyze_text():
    """Endpoint to analyze a single text string (expects JSON with key 'text')."""
    try:
        data = request.get_json() or {}
        text = data.get('text', '').strip()

        if not text:
            return jsonify({'error': 'No text provided'}), 400

        sentiment, translation, error = analyze_sentiment_spanish(text)

        if error:
            return jsonify({'error': error}), 500

        emotion, category = interpret_feeling(sentiment)

        result = {
            'original_text': text,
            'translated_text': translation,
            'analysis': {
                'negative': f"{sentiment['neg'] * 100:.1f}%",
                'neutral': f"{sentiment['neu'] * 100:.1f}%",
                'positive': f"{sentiment['pos'] * 100:.1f}%",
                'compound': sentiment['compound'],
                'sentiment': emotion,
                'category': category
            }
        }

        return jsonify(result)

    except Exception as e:
        return jsonify({'error': str(e)}), 500

In [9]:
@app.route('/analyze-excel', methods=['POST'])
def analyze_excel():
    """Endpoint to analyze an uploaded Excel file (multipart form-data, key 'file')."""
    try:
        if 'file' not in request.files:
            return jsonify({'error': 'No file uploaded'}), 400

        file = request.files['file']

        if file.filename == '':
            return jsonify({'error': 'No file selected'}), 400

        if not file.filename.lower().endswith(('.xls', '.xlsx')):
            return jsonify({'error': 'File must be an Excel file (.xls or .xlsx)'}), 400

        # Save the uploaded file to a temporary location
        with tempfile.NamedTemporaryFile(delete=False, suffix='.xlsx') as temp_file:
            file.save(temp_file.name)
            temp_path = temp_file.name

        # Analyze the file using the existing function
        results, error = analyze_excel_file(temp_path)

        # Clean up the temporary file
        try:
            os.unlink(temp_path)
        except Exception:
            pass

        if error:
            return jsonify({'error': error}), 500

        # Build summary
        total_analyzed = len(results)
        categories = {
            'positive': len([r for r in results if r.get('category') == 'positive']),
            'negative': len([r for r in results if r.get('category') == 'negative']),
            'neutral': len([r for r in results if r.get('category') == 'neutral'])
        }

        return jsonify({
            'total_analyzed': total_analyzed,
            'category_summary': categories,
            'results': results
        })

    except Exception as e:
        return jsonify({'error': str(e)}), 500

In [10]:
@app.route('/download-template')
def download_template():
    """Download an example Excel template containing sample text rows and instructions."""
    # Create a sample DataFrame with example comments/reviews
    example_data = {
        'Comments': [
            "I love this product, it's excellent!",
            "I'm not satisfied with the quality",
            "The service was acceptable, but could improve",
            "Amazing! I will definitely recommend it",
            "Doesn't meet my expectations"
        ],
        'Reviews': [
            "The product arrived on time and in good condition",
            "Poor customer service, very disappointed",
            "Works correctly, serves its purpose",
            "Terrible quality, I don't recommend it",
            "Excellent value for money"
        ]
    }

    df = pd.DataFrame(example_data)

    # Create an in-memory Excel file with a data sheet and an instructions sheet
    output = BytesIO()
    with pd.ExcelWriter(output, engine='openpyxl') as writer:
        df.to_excel(writer, sheet_name='Data', index=False)

        # Add an instructions sheet (English)
        instructions = pd.DataFrame({
            'Instructions': [
                '1. Add your text in any column',
                '2. Columns with long text will be detected automatically',
                '3. Save the file and upload it to the application',
                '4. The system will analyze all detected texts automatically',
                '5. You may use multiple columns if needed'
            ]
        })
        instructions.to_excel(writer, sheet_name='Instructions', index=False)

    output.seek(0)

    return send_file(
        output,
        mimetype='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet',
        as_attachment=True,
        download_name='sentiment_analysis_template.xlsx'
    )

In [11]:
@app.route('/health')
def health_check():
    """Health check endpoint for the service."""
    return jsonify({
        'status': 'running',
        'service': 'Sentiment Analyzer',
        'version': '1.0'
    })

In [ ]:
if __name__ == '__main__':
    print("🚀 Sentiment Analyzer server started!")
    print("📊 Available endpoints:")
    print("   - GET  / -> Web interface")
    print("   - POST /analyze -> Analyze single text")
    print("   - POST /analyze-excel -> Analyze Excel file")
    print("   - GET  /download-template -> Download template")
    print("   - GET  /health -> Service health")
    print("\n📍 Server running at: http://localhost:5000")
    # run without the reloader in notebooks to prevent SystemExit from the reloader
    app.run(debug=True, host='0.0.0.0', port=5000, use_reloader=False)

🚀 Sentiment Analyzer server started!
📊 Available endpoints:
   - GET  / -> Web interface
   - POST /analyze -> Analyze single text
   - POST /analyze-excel -> Analyze Excel file
   - GET  /download-template -> Download template
   - GET  /health -> Service health

📍 Server running at: http://localhost:5000
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.80.14:5000
Press CTRL+C to quit
127.0.0.1 - - [23/Sep/2025 20:55:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Sep/2025 20:55:20] "POST /analyze-excel HTTP/1.1" 200 -


In [ ]:
%tb

SystemExit: 1